# Importing necessary libraries

In [1]:
import yfinance as yf
import psycopg2
from psycopg2 import sql
import datetime
import random
import pandas as pd
import holidays
from datetime import timedelta

# Connect to PostgreSQL database


In [2]:
def connect_to_db():
    try:
        conn = psycopg2.connect(
            host="localhost",  
            database="stock_data",  
            user="umutknsk",  
            password="umut1234"  
        )
        cur = conn.cursor()
        return conn, cur
    except Exception as e:
        print(f"Error connecting to database: {e}")

# SQL query to insert data into the database

In [3]:
def get_insert_query():
    return sql.SQL("""
        INSERT INTO stock_table (symbol, name, open, high, low, close, volume, dividends, stock_splits, timestamp)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (symbol, timestamp) DO NOTHING;
    """)

# Function to process each row of stock data

In [4]:
def process_row(row, symbol):
    name = row.get('shortName', 'Unknown Name')  
    open_price = row['Open'] if pd.notna(row['Open']) else None  
    high = row['High'] if pd.notna(row['High']) else None  
    low = row['Low'] if pd.notna(row['Low']) else None  
    close = row['Close'] if pd.notna(row['Close']) else None 
    volume = row['Volume'] if pd.notna(row['Volume']) else None  
    dividends = row['Dividends'] if pd.notna(row['Dividends']) else None 
    stock_splits = row['Stock Splits'] if pd.notna(row['Stock Splits']) else None  
    timestamp = row.name  
    return (symbol, name, open_price, high, low, close, volume, dividends, stock_splits, timestamp)

# Function to fetch and process stock data

In [5]:
def fetch_and_process_stock_data(symbol, conn, cur, insert_query):
    stock = yf.Ticker(symbol)
    stock_info = stock.history(period='max')

    if stock_info.empty:
        print(f"No stock data available for {symbol}.")  
        return None, None, None, None

    stock_info.index = stock_info.index.tz_localize(None)  

    first_date = stock_info.index.min()  
    last_date = stock_info.index.max()  
    start_date = first_date.replace(tzinfo=None)  

    print(f"First date is {first_date} for {symbol}")

    delta_days = (last_date - start_date).days  
    if delta_days <= 0:
        return None, None, None, None

    us_holidays = holidays.US(years=range(first_date.year, last_date.year + 1))  

    return stock_info, start_date, delta_days, us_holidays


# Function to handle partially removed dates and update stock data

In [ ]:
def handle_partially_removed_dates(symbol, stock_info, start_date, delta_days, us_holidays, random_dates_dict):
    random.seed(42)
    
    fully_removed_dates = set()  
    while len(fully_removed_dates) < min(10, delta_days):  
        random_days = random.randint(0, delta_days)
        random_date = start_date + timedelta(days=random_days)

        if random_date.weekday() < 5 and random_date.date() not in us_holidays:
            fully_removed_dates.add(random_date.date())

    print(f"Selected dates (excluded) for {symbol}: {list(fully_removed_dates)}")

    partially_removed_dates = set()  
    while len(partially_removed_dates) < min(10, delta_days):  
        random_days = random.randint(0, delta_days)
        random_date = start_date + timedelta(days=random_days)

        if (random_date.weekday() < 5 and random_date.date() not in us_holidays and 
            random_date.date() not in fully_removed_dates):
            partially_removed_dates.add(random_date.date())
    
    print(f"Partially removed dates for {symbol}: {list(partially_removed_dates)}")

    random_dates_dict[symbol] = list(fully_removed_dates)

    partially_removed_dates = pd.to_datetime(list(partially_removed_dates))  
    
    for date in partially_removed_dates:
        if date in stock_info.index:
            num_columns_to_nan = random.randint(1, 4)  
            columns_to_nan = random.sample(["Open", "High", "Low", "Close", "Volume"], num_columns_to_nan)
            stock_info.loc[stock_info.index == date, columns_to_nan] = None
            
    print(f"Updated stock data for {symbol}:\n", stock_info.loc[stock_info.index.isin(partially_removed_dates)])
    return stock_info

# Function to insert data into the database

In [7]:
def insert_data_into_db(symbol, stock_info_filtered, cur, insert_query, random_dates_dict):
    stock_info_filtered['Date'] = stock_info_filtered.index.date
    stock_info_filtered = stock_info_filtered[~stock_info_filtered['Date'].isin(random_dates_dict[symbol])]

    if stock_info_filtered.empty:
        print(f"No valid data to insert for {symbol}.")
        return

    rows_to_insert = stock_info_filtered.apply(lambda row: process_row(row, symbol), axis=1).tolist()
    cur.executemany(insert_query, rows_to_insert)

# Main Process

In [8]:
conn, cur = connect_to_db()
insert_query = get_insert_query()

symbols = ['AAPL', 'NVDA', 'SOUN', 'SMCI']
random_dates_dict = {symbol: [] for symbol in symbols}  # Initialize the dictionary for random dates

for symbol in symbols:
    stock_info, start_date, delta_days, us_holidays = fetch_and_process_stock_data(symbol, conn, cur, insert_query)
    
    if stock_info is None:
        continue
    
    stock_info = handle_partially_removed_dates(symbol, stock_info, start_date, delta_days, us_holidays, random_dates_dict)
    insert_data_into_db(symbol, stock_info, cur, insert_query, random_dates_dict)

# Commit the transaction after inserting all data
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

print("All data inserted successfully!")

First date is 1980-12-12 00:00:00 for AAPL
Selected dates (excluded) for AAPL: [datetime.date(1987, 3, 17), datetime.date(1982, 1, 25), datetime.date(2014, 3, 18), datetime.date(1993, 4, 14), datetime.date(1990, 12, 17), datetime.date(1985, 12, 10), datetime.date(2009, 8, 18), datetime.date(2014, 3, 4), datetime.date(2011, 4, 21), datetime.date(1985, 7, 18)]
Partially removed dates for AAPL: [datetime.date(1999, 11, 15), datetime.date(1982, 4, 14), datetime.date(1984, 11, 5), datetime.date(1991, 5, 20), datetime.date(2007, 12, 14), datetime.date(2003, 8, 13), datetime.date(2007, 6, 8), datetime.date(1990, 10, 3), datetime.date(1982, 2, 19), datetime.date(2006, 2, 14)]
Updated stock data for AAPL:
                 Open      High       Low     Close       Volume  Dividends  \
Date                                                                         
1982-02-19  0.064816  0.064816       NaN       NaN          NaN   0.000000   
1982-04-14  0.055373  0.055802       NaN       NaN  1135904